# Tokenization

Whenever you explain how the LLM works, you usually start that it can't actually operate on *text*. It operates on the sequence of vectors but before that on the sequence of integer numbers (encoding the token ID). We can then trivially get a vector by just passing the integers through `nn.Embedding`. So, the text needs to be transformed into the sequence of numbers. For this, we need a tokenizer.

## BPE

Let's skip the old word or rule-based methods and jump straight into Byte-Pair Encoding (BPE). We'll skip for now how it's trained and focus on the inference.

BPE has a vocabulary that matches a sequence of **bytes** to a token ID. So, when you have a Python string, it (more or less) just does `s.encode("utf-8")` and then tries to greedily match the bytes to the longest possible tokens.

Let's implement it. First, we need to have a way to store these tokens. The problem is that if we want to store a token `\t`, if you just call `print(token)`, the result will be invisible. Even worse, some of the bytes are weird unprintable control characters if you just assume they are unicode symbols. So, the first step, wee need to match bytes (practically numbers from 0 to 255) to some nice printable characters.

Most of the implementations use GPT-2 convension for it:
1. All the bytes that correspond to ASCII punctuation and alphanumeric characters are mapped to themselves (the bytes from `!` to `~`)
2. The Latin-1 supplement from `¡` to `¬` is also matched to itself.
3. Also, another block of Latin-1 from `®` to `ÿ` is matched to itself. This ensures that all roughly normal "visible symbols" are matched to themselves. Nice. The tricky part is the next.
4. Whenever you have a byte outside these characters, it is matched to the character corresponding to 256 + the number of these "weird" characters we've seen already when iterating from 0 to 255. This also ensures that whitespace, `\t`, `\n`, etc are mapped to some "safe range of printable stuff".

In [ ]:
def bytes_to_unicode() -> dict[int, str]:
    """
    Creates an mapping from integers to `str` implementing the logic we described above
    """

    # your code goes here


def unicode_to_bytes() -> dict[str, int]:
    """
    Inverse transformation
    """
    # your code goes here


B2U = bytes_to_unicode()
U2B = unicode_to_bytes()


def encode_bytes_to_visible(s: bytes) -> str:
    return "".join(B2U[b] for b in s)


def decode_visible_to_bytes(s: str) -> bytes:
    return bytes(U2B[ch] for ch in s)

In [ ]:
assert isinstance(B2U, dict)
assert len(B2U) == 256
assert B2U[0] == "Ā"
assert B2U[20] == "Ĕ"
assert B2U[65] == "A"
assert B2U[255] == "ÿ"
assert encode_bytes_to_visible(" token".encode("utf-8")) == "Ġtoken"
assert encode_bytes_to_visible("\ntoken".encode("utf-8")) == "Ċtoken"

## Normalization

The course was written in 2025 and at that time no other normalization except [NFC](https://en.wikipedia.org/wiki/Unicode_equivalence#Normalization) exists. Gotta be honest, a very Unicode-specific stuff that didn't dive too deep in but luckly it's literally one Python line.

In [ ]:
import unicodedata


def nfc_normalize(text: str) -> str:
    return unicodedata.normalize("NFC", text)

## Pre-Tokenization

Pre-tokenization is relatively straighforward but mostly overlooked process. This is mainly the source of difference between different models' BPE-based tokenizers. Pre-tokenization is doing the following: it takes the **text** (Python `str`) and splits it into *pre-tokens* (a list of `str`).

The actual BPE is then applied only inside these pre-tokens and the result is then concatenated. This is done both when running the tokenizer and when training it. We haven't discussed the training process yet but it actually defines what merges we allow. If the pre-tokenizer always splits by the whitespace, the learned tokens will never contain more than one word (in English, German, Dutch, Serbian, Greek, Russian and many (all?) other Indo-European languages meaning. For, let's say, Chineese the result might be very different! In some sense, for Chineese, the standard GPT-2 BPE is kinda similar to sentencepiece) because two words will always end up in different pre-tokens, so they will be tokenized independently.

The basic pre-tokenizer might just split by whitespace (but the whitespace must be preserved in the resulting element, so `str.split(" ")` isn't a valid pre-tokenizer). It might be just `lambda s: [s]`, so we allow all the possible merges. But most often than not the pre-tokenizer is defined by the regexp. Here are some of them:

In [ ]:
GPT2_PRETOKENIZE_REGEXP = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_PRETOKENIZE_REGEXP = (
    r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
)
QWEN_PRETOKENIZE_REGEXP = r"""(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"""

I really suggest to play around with them [here](https://regex101.com/r/NRYk93/1).

The problem with pre-tokenizers is literally because these regexps (regexp problem in general) are unreadable. Below is a decent explanation from GPT-5 Thinking of what is going on in Qwen regexp:


### What a “pre-tokenization regex” is

Before BPE/Unigram does subword merges, most modern tokenizers first **split the raw string into chunks** (words, numbers, punctuation, spaces, newlines, contractions, etc.). That split is driven by a **single big regex**. The subword algorithm then runs **inside each chunk** to produce final token IDs. So this regex determines what the model even has a chance to merge—change it and you change what statistics the BPE learns.

Your pattern:

```
PRETOKENIZE_REGEX =
(?i:'s|'t|'re|'ve|'m|'ll|'d)
|[^\r\n\p{L}\p{N}]?\p{L}+
|\p{N}
| ?[^\s\p{L}\p{N}]+[\r\n]*
|\s*[\r\n]+
|\s+(?!\S)
|\s+
```

This is an **ordered OR** (alternation). Engines match **left to right**, first thing that fits wins. `\p{L}` = any Unicode letter, `\p{N}` = any Unicode number. Comments below reference **branches** in order.

#### 1) `(?i:'s|'t|'re|'ve|'m|'ll|'d)`

* **Meaning:** Contraction suffixes, case-insensitive, as atomic tokens.
* **Why first:** You want `"I'm"` → `I` + `’m`, `"CAN'T"` → `CAN` + `’T`, not one big “word”.
* **Examples:**

  * `"It’s"` → `It` + `’s`
  * `"YOU’LL"` → `YOU` + `’LL`

#### 2) `[^\r\n\p{L}\p{N}]?\p{L}+`

* **Meaning:** A **word** (`\p{L}+`) optionally **prefixed by one non-letter/number that’s not CR/LF** (often opening punctuation).
* **Effect:** Binds an opening mark to the word: `“Hello` is one chunk, the closing `”` will be handled later.
* **Examples:**

  * `“Hello` → one chunk (leading `“` + `Hello`)
  * `(world` → one chunk
  * `foo` → one chunk

#### 3) `\p{N}`

* **Meaning:** **One** numeric character.
* **Design choice:** Single-digit chunks let BPE learn frequent multi-digit merges statistically (`2025` becomes `2|0|2|5` pre-BPE, then merges to `2025` if common). If this were `\p{N}+`, the whole number would be a pretoken chunk and BPE would have less leverage.
* **Examples:** `2025` → `2 | 0 | 2 | 5` (four chunks at the pre-token stage)

#### 4) ` ?[^\s\p{L}\p{N}]+[\r\n]*`

* **Meaning:** Optional leading space, then **one or more symbols/punct that are neither letters, numbers, nor whitespace**, then optional CR/LF.
* **Effect:** Groups runs like `—–…`, `!!!`, emoji sequences, or `:)`, and **captures a following newline** if present (useful for end-of-line punctuation).
* **Examples:**

  * `" —"` → one chunk (leading space + em dash)
  * `"!!!\n"` → one chunk (bang-run + newline)
  * `"🙂👍"` → one chunk

#### 5) `\s*[\r\n]+`

* **Meaning:** Blank-line / line-break chunk: optional spaces then one or more CR/LF.
* **Why separate from 4):** This catches **pure** line breaks (including spaces before them) when there isn’t a symbol run just before.
* **Examples:**

  * `"\n"` or `"   \r\n"` → one chunk

#### 6) `\s+(?!\S)`

* **Meaning:** Whitespace **not followed by a non-space** → trailing spaces at end of line/string.
* **Why:** Trailing spaces get isolated so they don’t glom onto the next word on the next line.
* **Example:** `"end   \n"` → the `   ` before `\n` is one chunk here.

#### 7) `\s+`

* **Meaning:** Any other whitespace (the general “leftovers” case).
* **Examples:** Inter-word spaces when none of the earlier branches applied.

---

#### How a line is split (walkthrough)

Input:
`He said, “It’s 2025—finally!”\n\nOK.`

Pre-token chunks (conceptually):

1. `He` (branch 2)
2. ` ` (branch 7)
3. `said` (2)
4. `,` (4)
5. ` ` (7)
6. `“It` (2; leading open quote bound to word)
7. `’s` (1; contraction)
8. ` ` (7)
9. `2 | 0 | 2 | 5` (3,3,3,3)
10. `—finally` (2; em dash bound to word because of the optional leading non-letter)
11. `!”` (4; symbol run)
12. `\n` (5)
13. `\n` (5)
14. `OK` (2)
15. `.` (4)

BPE then merges inside these pieces (e.g., `'s`, common words, frequent digit sequences, frequent emoji runs, etc.).

> Note on engines: `\p{…}` classes and that scoped `(?i: … )` are supported in PCRE/Oniguruma/Rust-regex/`regex` (PyPI). Python’s built-in `re` **does not** support `\p{L}/\p{N}`.

Ok, enough yapping, let's implement this one-liner function:

In [ ]:
import regex as re  # note that it's not the standard re. You need to actually add it to dependencies


def pretokenize(text: str, pattern: str = QWEN_PRETOKENIZE_REGEXP) -> list[str]:
    # your code goes here

In [ ]:
assert pretokenize(
    "...I know he dyin' (oh my, oh my God) 6-7, I just bipped right on the highway (Bip, bip)", QWEN_PRETOKENIZE_REGEXP
) == [
    "...",
    "I",
    " know",
    " he",
    " dyin",
    "'",
    " (",
    "oh",
    " my",
    ",",
    " oh",
    " my",
    " God",
    ")",
    " ",
    "6",
    "-",
    "7",
    ",",
    " I",
    " just",
    " bipped",
    " right",
    " on",
    " the",
    " highway",
    " (",
    "Bip",
    ",",
    " bip",
    ")",
]

## Actual BPE Tokenization

### Training

On a high level, BPE does the following: we start with a set of 256 tokens which are raw bytes. Then, we calculate the number of all pairs of tokens in the training corpus. After we've done it, we take the pair with the most occurences in the dataset, merge it into a new token, and calculate the pairs again. Repeat until we can't merge anything or we reached the required vocabulary size. The training process might be explained with more details but actually this is so intuitive already so I don't know what else to write. *Remember that tokenization is done inside the byte-encoded pre-tokens.*

As a result, we have a vocabulary of tokens and the list of merges of tokens in chronological order when they happened. These are the `vocab.json` and `merges.txt` from HuggingFace.

### Inference

We take a list of pre-tokens. For each pre-token, we first split it into the set of "tokens" which are bytes, similarly to training. Then, we find all the pairs of tokens that exist in the merges, so they can be merged into a single token. There can be many of them, so which one to actually apply? If you can think about it, it will highly affect the result. Let's say we want to tokenize the word `and`. We start with `("a", "n", "d")`. What if we have both `("a", "n")` and `("n", d")` merges? The result would be different depending on the path we take. So, to make the tokenization stable, we take **the earliest merge** that happened during training. Remember, we support it during the training procedure. Then, we repeat the process until we can't merge anything anymore.

The result for the whole text is just a concatenation of results for all the pre-tokens.

Let's implement it!

We will do proper typings, so introducing these makes the code a bit more readable IMO

In [ ]:
Pair = tuple[str, str]
Token = tuple[
    str, ...
]  # a pre-token that is represented as a set of "symbols" that at first are bytes but then will become actual tokens

Some helper functions that will allow us to write a readable implementation

In [ ]:
from collections import Counter


def get_pairs(token: Token) -> set[Pair]:
    """
    Given a pre-token as a tuple of "symbols" (which at first are visible byte-chars), calculate the set of all the pairs of them.
    """
    return {(token[i], token[i + 1]) for i in range(len(token) - 1)}


def compute_pair_stats(corpus: list[Token]) -> Counter[Pair]:
    """
    Reurns a counter with frequencies of all pairs of symbols in the corpus
    """
    stats: Counter[Pair] = Counter()
    for token in corpus:
        stats.update(get_pairs(token))
    return stats


def merge_token(token: Token, pair: Pair, new_symbol: str) -> Token:
    """
    Given a pre-token as tuple of symbols and a pair of two symbols that will become a new token,
    replaces occurances when they are together on after another with a new symbol
    """
    # your code goes here


def replace_best_pair(corpus_tokens: list[Token], best_pair: Pair, symbol: str) -> list[Token]:
    """
    Replaces a pair of tokens in all the pre-tokens in the training corpus with a new token.
    God I hate this terminology.
    """
    return [merge_token(token, best_pair, symbol) if len(token) >= 2 else token for token in corpus_tokens]


assert merge_token(("a", "b", "c", "b"), ("b", "c"), "bc") == ("a", "bc", "b")

BPE inference! Functional way by passing the vocab and merges as function arguments.

In [ ]:
def get_best_pair(pairs: set[Pair], merge_ranks: dict[Pair, int]) -> Pair | None:
    """
    Finds the pair with a minimum rank in merges, so we can replace a pair of tokens with one token
    """
    # your code goes here


def bpe_inference(pre_token: str, vocab: dict[str, int], merge_ranks: dict[Pair, int]) -> Token:
    """
    Given a byte-encoded pre-token, run BPE inference on it to get a list of tokens out of it
    """
    if not pre_token:
        return []

    word: Token = tuple(pre_token)
    if len(word) == 1:
        return [word[0]]

    while True:
        pairs = ...
        best_pair = ...
        if best_pair is None:
            break

        new_symbol = ...
        word = ...

    return word


assert bpe_inference("abcd", {"a": 0, "b": 1, "c": 2, "d": 3, "bc": 4}, {("b", "c"): 0}) == ("a", "bc", "d")

BPE training and a a HuggingFace-like interface for the tokenization.

In [ ]:
from tqdm.auto import tqdm
import os
import json


class BPETokenizer:
    def __init__(
        self,
        vocab: dict[str, int] | None = None,
        merge_ranks: dict[Pair, int] | None = None,
        pretokenizer_regexp: str = QWEN_PRETOKENIZE_REGEXP,
    ):
        self.vocab = vocab
        self.merge_ranks = merge_ranks
        self.pretokenizer_regexp = pretokenizer_regexp

    def train(self, texts: list[str], vocab_size: int, is_testing: bool = False) -> "BPETokenizer":
        """
        Trains a BPE
        """
        # 1) Prepare initial "corpus" at byte-level (visible Unicode chars) per pretokenized piece
        corpus: list[Token] = []
        for text in texts:
            text_normalized = ...
            # your code goes here

        if is_testing:
            assert corpus == [
                "A",
                "Ġsentence",
                "A",
                "Ġ",
                "6",
                "9",
                "Ġnumber",
                "ðĿĶĺðĿĶ«ðĿĶ¦ðĿĶłðĿĶ¬ðĿĶ¡ðĿĶ¢",
                "ĠOK",
                ".",
                "Tabs",
                "ĉand",
                "Ġnewlines",
                "Ċ",
                "are",
                "Ġalso",
                "Ġtokens",
                ".",
            ]

        # 2) Initialize vocab with 256 byte symbols + (optional) special tokens
        self.vocab: dict[str, int] = {}
        next_token_id = 0

        # Add 256 visible-byte symbols (as one-char strings)

        # your code goes here

        if is_testing:
            assert len(self.vocab) == 256
            assert self.vocab["Ġ"] == 32

        # Merge tokens until we reach the vocabulary size
        step = 0
        self.merge_ranks: dict[Pair, int] = {}

        progress = tqdm()

        while next_token_id < vocab_size:
            pair_stats = ...
            if not pair_stats:
                break  # cannot merge further (all tokens are singletons)

            # Deterministic best: highest freq, tie-break lexicographically
            max_freq = max(pair_stats.values())
            candidate_pairs = [p for p, c in pair_stats.items() if c == max_freq]
            best_pair = min(candidate_pairs)  # stable tie-break
            new_symbol = "".join(best_pair)  # concatenation of the two symbols

            if new_symbol not in self.vocab:
                # your code goes here
            else:
                # I actually haven't given a thought on whether this can indeed happen or not. It seems it shouldn't
                self.merge_ranks[best_pair] = len(self.merge_ranks)

            # Replace the pair with a new token in the corpus
            corpus = ...
            step += 1
            progress.update(1)
            progress.set_description(f"Merges: {step}, vocab={next_token_id}/{vocab_size}")

        progress.close()
        if is_testing:
            assert self.vocab["Ċ"] == 10
            assert self.vocab["Ġalso"] == 315
            assert self.vocab["Ġnumber"] == 319

        return self

    def encode(self, text: str) -> list[int]:
        """
        Encodes a normal text into token IDs
        """
        # Normalize
        text = nfc_normalize(text)

        ids: list[int] = []

        # Pre-tokenize
        pre_tokens = ...
        for pre_token in pre_tokens:
            # Encode the pre-token to visible representation of bytes
            encoded = ...
            # Run BPE to merge tokens
            symbols = ...
            for symbol in symbols:
                ids.append(self.vocab[symbol])
        return ids

    def decode(self, token_ids: list[int]) -> str:
        """
        Given a list of token IDs, returns a proper text they correspond to
        """
        inv_vocab = {i: s for s, i in self.vocab.items()}
        visible_bytes: list[str] = []
        for token_id in token_ids:
            ...
        byte_string = ...
        return byte_string.decode("utf-8", errors="strict")

    def save_pretrained(self, path: str) -> None:
        """
        Store the tokenizer in a HuggingFace-compatible BPE format. We loose pre-tokenizer this way unfortunately.
        Storing a regexp is too advanced for the HF ecosystem.
        """
        vocab_path = os.path.join(path, "vocab.json")
        merges_path = os.path.join(path, "merges.txt")

        with open(vocab_path, "w") as f:
            json.dump(self.vocab, f)

        with open(merges_path, "w") as f:
            f.write("\n".join([f"{a}\t{b}" for a, b in self.merge_ranks.keys()]))

In [ ]:
texts = [
    "A sentence",
    "A 69 number",
    "𝔘𝔫𝔦𝔠𝔬𝔡𝔢 OK.",
    "Tabs\tand newlines\nare also tokens.",
]

In [ ]:
tokenizer = BPETokenizer()
tokenizer.train(texts, 320, is_testing=True)

In [ ]:
assert tokenizer.encode("A sentence with a number") == [65, 316, 32, 119, 105, 116, 104, 32, 97, 319]
assert tokenizer.decode([65, 316, 32, 119, 105, 116, 104, 32, 97, 319]) == "A sentence with a number"
s = "This must ALWAYS work"
assert tokenizer.decode(tokenizer.encode(s)) == s